In [1]:
import json
import os
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from docx import Document
from docx.shared import Pt
import pickle
import os
from tqdm import tqdm
import time
from langdetect import detect
import matplotlib.pyplot as plt
import collections
from IPython.display import display

import spacy.cli
from spacy.language import Language
import math

from IPython.display import display

In [2]:
path_dataset = "../dataset/"

In [3]:
flusso_unificato = pd.read_csv(path_dataset+"flusso_TEDx_completo_unificato.csv", sep = '\t')

In [4]:
flusso_ripulito = pd.read_csv(path_dataset+"flusso_TEDx_completo_ripulito.csv", sep = '\t')

In [5]:
flusso_unificato

,Original_sentences
0,"C'è una cosa che io faccio sempre, quando pren..."
1,All'atterraggio e al decollo io guardo fuori d...
2,"Probabilmente lo fate anche voi, anche quando ..."
3,"Perché in realtà, una delle cose che ci piace ..."
4,Gli astronauti quando sono nello spazio per la...
...,...
51889,E una persona che ha questa capacità non può n...
51890,E allora come faccio io per cambiare il mondo?\n
51891,"Ci sono molti che vogliono cambiare il mondo, ..."
51892,Io credo che se si vuole veramente cambiare il...


In [6]:
flusso_ripulito

,Sentences
0,"C'è una cosa che io faccio sempre, quando pren..."
1,All'atterraggio e al decollo io guardo fuori d...
2,"Probabilmente lo fate anche voi, anche quando ..."
3,"Perché in realtà, una delle cose che ci piace ..."
4,Gli astronauti quando sono nello spazio per la...
...,...
50089,"Allora, essere ottimisti forse oggi è semplice..."
50090,E una persona che ha questa capacità non può n...
50091,E allora come faccio io per cambiare il mondo?
50092,"Ci sono molti che vogliono cambiare il mondo, ..."


In [7]:
for frase in flusso_unificato.itertuples():
    if '[' in frase.Original_sentences or ']' in frase.Original_sentences:
        print(frase.Original_sentences)

(Risate) [Livello di risoluzione nell'analisi del DNA] E l'analisi del DNA, in realtà, ci ha permesso di aumentare l'efficienza della selezione.

Questa mutazione è presente in alcune varietà: quindi quello che fa l'editing, essenzialmente, [è riuscire] a copiare le soluzioni che sono state trovate dalla selezione naturale che sono presenti nelle razze locali e nelle varietà locali.

Per cui [possono] catturare più CO2 dall'atmosfera e ridurre gli impatti ambientali.

Ho pensato ai pericoli di quella che chiamiamo l’information overload, cioè il fatto che siamo immersi in montagne di dati e di informazioni e rischiamo di confondere i dati e le informazioni con la conoscenza, [che] son due cose diverse.

L'esperienza è un grande valore, si trovano dimensioni di saggezza in persone che magari non sono mai state nella Silicon Valley, o forse, [proprio] perché non ci sono state hanno una dimensione di saggezza.

La Terra è a [11 milioni di km].

In realtà, la biodiversità del suolo però è 

In [8]:
for frase in flusso_unificato.itertuples():
    if b"\xe2" in frase.Original_sentences.encode('utf-8') and 'Musica' in frase.Original_sentences:
        print(frase.Original_sentences)

(Voci in inglese in sottofondo) (Fine delle voci in inglese in sottofondo) (Musica e voci indistinte) (Fine musica) (Applausi) "Non avrebbe potuto essere lavato mai più; avrebbe dimostrato che l’uomo, il genere umano, noi insomma, eravamo potenzialmente capaci di costruire una mole infinita di dolore; e che il dolore è la sola forza che si crei dal nulla, senza spesa e senza fatica.

(Musica) Qui In Italia c’è un bell'uomo Che vive a Roma, che vive a Roma!

(Esultazione) (Musica) Alessia Zecchini - 107m FIM WORLD RECORD DEANS BLUE HOLE, BAHAMAS - 26/07/18 (Applausi) Quando mi chiedono perché amo l’apnea, penso a queste tre parole: profondità, impegnativo, ed entusiasmante.

(Applausi) (Musica di chitarra) ♪ Il venditore afgano ha visto esploder gambe ♪ ♪ Anche di suo fratello e odia perché ne ha bisogno ♪ ♪ Il giovane italiano pensa che sia un coglione ♪ ♪ Che questo non ha senso, non è valida ragione ♪ ♪ Peccato che domenica ha infilato sei centimetri di lama ♪ ♪ Dentro a un petto che

In [9]:
for frase in flusso_unificato.itertuples():
    if "<" in frase.Original_sentences:
        print(frase.Original_sentences)

Quello che fa un artista di computer grafica, normalmente, è aprire il suo computer, aprire Maya che è un software come gli altri, 3D Studio Max o altri, schiaccia un bottone con scritto <i>Create</i>, dopo questo bottone schiaccia <i>Polygon</i>, <i>Cube</i> e <i>Muovi Vertice</i> - sono comandi veri del software; lo rifà un milione di volte, tutto il giorno rifà questa cosa, e alla fine ottiene questo.

Milioni di persone hanno <i>deciso</i> di guardare la pubblicità.

<i>Noi</i> li abbiamo chiamati social media, perché chiamarli social media significava, in qualche modo, utilizzarli come media.

Se ci fosse qui uno speaker del TED di quelli - direbbe <i>meaningful</i>.

Ma qualcuno mi obietta: no, ma ho letto che non è il Bitcoin, è la <i>blockchain</i>, la tecnologia sottostante Bitcoin, a essere fondamentale.

Però Walter non amava molto il lavoro di banca, e i suoi <i>leones</i> erano altri.

E quindi questa cosa ci sta portando verso un'altra situazione, verso un altro <i>leones

In [24]:
for frase in flusso_unificato.itertuples():
    if "risate" in frase.Original_sentences:
        print(frase.Original_sentences)

Non è solo l'aspetto, da Cesena, della sua maestosità, è l'odore, è il ricordo dell'odore che aveva l’aria quando ero un bambino, il sapore del cibo, le risate piene degli avventori del bar a cui mi reco, dopo mangiato, a prendere il caffè.

(Risate) Vedete, noia e tempestività, uguale una tempesta di risate.

Guardate gli occhi... cioè, però... (risate) Guardate verso l'alto con gli occhi, ecco, che aiuta a cambiare le onde celebrari verso il positivo e nel frattempo aprite la bocca sorridendo sempre, senza fare "Aaa" che non stiamo dal medico adesso.

La sua storia, il suo sangue, la sua intelligenza, le sue risate, la sua forza, dov'è?

Qualcuno non avrebbe niente da ridere, eppure ti fa spanciare dalle risate; ironico, allegro, divertente.

Qualche altra persona, invece, ti fa scompisciare dalle risate anche se, in realtà, di motivi per ridere non ne avrebbe molti.

Io ero lì che mi sbudellavo dalle risate, allora mi dava i pugni.

Ma una cosa ho capito nel tempo: che a farmi stare

In [45]:
for frase in flusso_unificato.itertuples():
    if "R." in frase.Original_sentences:
        print(frase.Original_sentences)

CRISPR.

Altra cosa ancora: il SAR.

Quello era il SAR.

Ci sono tanti altri prodotti che possono essere ottenuti con CRISPR.

Oppure anche le patate possono essere modificate; ma tanti, tanti altri prodotti ogni giorno escono sui nostri articoli scientifici e dimostrano le potenzialità di CRISPR.

E poi, finalmente, è arrivata CRISPR.

Ed è questa la rivoluzione che ha portato CRISPR.

Un passo fondamentale sarà quindi ITER.

A.R.: La Comunicazione Aumentativa, poi ne parleremo, è rivolta a quei bambini, o adulti, che per cause congenite - cioè sono persone che hanno avuto un'asfissia alla nascita e hanno una paralisi cerebrale infantile; sono le persone che vediamo in carrozzina che spesso non riescono neanche a muovere le mani e ad avere alcuna autonomia; persone con un ritardo cognitivo notevole, spesso dovuto a cause genetiche, malattie genetiche che provocano ritardo intellettivo e, per molte delle persone che ne sono affette, impossibilità a parlare.

A.R.: La Comunicazione Aume

In [10]:
def cerca_frasi_ripulite(stringa):
    
    for frase in flusso_ripulito.itertuples():
        if stringa in frase.Sentences:
            return frase.Sentences
            break

In [12]:
"I comportamenti problema[tici] sono diminuiti e abbiamo potuto iniziare a instaurare una relazione, a conoscerci."

'I comportamenti problema[tici] sono diminuiti e abbiamo potuto iniziare a instaurare una relazione, a conoscerci.'

In [13]:
cerca_frasi_ripulite("I comportamenti")

'I comportamenti problematici sono diminuiti e abbiamo potuto iniziare a instaurare una relazione, a conoscerci.'

In [14]:
for frase in flusso_ripulito.itertuples():
    if "Non avrebbe potuto essere lavato mai più" in frase.Sentences:
        print(frase.Sentences)

"Non avrebbe potuto essere lavato mai più; avrebbe dimostrato che l'uomo, il genere umano, noi insomma, eravamo potenzialmente capaci di costruire una mole infinita di dolore; e che il dolore è la sola forza che si crei dal nulla, senza spesa e senza fatica.


In [15]:
def popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi):
    
    lista_frasi.append((frase_sporca, frase_ripulita))
    
    return lista_frasi

In [16]:
lista_frasi = []

In [17]:
frase_sporca = "(Applausi) (Musica di chitarra) ♪ Il venditore afgano ha visto esploder gambe ♪ ♪ Anche di suo fratello e odia perché ne ha bisogno ♪ ♪ Il giovane italiano pensa che sia un coglione ♪ ♪ Che questo non ha senso, non è valida ragione ♪ ♪ Peccato che domenica ha infilato sei centimetri di lama ♪ ♪ Dentro a un petto che batteva per un'altra squadra ♪ ♪ Che ha incrociato la sua strada e non doveva ♪ ♪ Che sgarro imperdonabile ♪ ♪ E Irina vende il corpo, ma solo più da tergo ♪ ♪ Non vuole occhi davanti a far da specchio del suo inferno ♪ ♪ E tu che a malapena hai il tuo smalto da abbinare ♪ ♪ Pensi di capire?"
frase_ripulita = cerca_frasi_ripulite("Il venditore afgano ha visto esploder gambe")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [18]:
frase_sporca = "I comportamenti problema[tici] sono diminuiti e abbiamo potuto iniziare a instaurare una relazione, a conoscerci."
frase_ripulita = cerca_frasi_ripulite("I comportamenti")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [19]:
lista_frasi

[("(Applausi) (Musica di chitarra) ♪ Il venditore afgano ha visto esploder gambe ♪ ♪ Anche di suo fratello e odia perché ne ha bisogno ♪ ♪ Il giovane italiano pensa che sia un coglione ♪ ♪ Che questo non ha senso, non è valida ragione ♪ ♪ Peccato che domenica ha infilato sei centimetri di lama ♪ ♪ Dentro a un petto che batteva per un'altra squadra ♪ ♪ Che ha incrociato la sua strada e non doveva ♪ ♪ Che sgarro imperdonabile ♪ ♪ E Irina vende il corpo, ma solo più da tergo ♪ ♪ Non vuole occhi davanti a far da specchio del suo inferno ♪ ♪ E tu che a malapena hai il tuo smalto da abbinare ♪ ♪ Pensi di capire?",
  "Il venditore afgano ha visto esploder gambe Anche di suo fratello e odia perché ne ha bisogno Il giovane italiano pensa che sia un coglione Che questo non ha senso, non è valida ragione Peccato che domenica ha infilato sei centimetri di lama Dentro a un petto che batteva per un'altra squadra Che ha incrociato la sua strada e non doveva Che sgarro imperdonabile E Irina vende il c

In [20]:
frase_sporca = "[idee con le ali] (musica) [anime e persone con le radici] (musica) (Applausi) Qualche anno fa ho avuto un'idea per un documentario che secondo me era un'idea geniale, e sono andata a presentarla ad un canale televisivo."
frase_ripulita = cerca_frasi_ripulite('canale televisivo')

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [21]:
frase_sporca = "No ragazzi ce ne vuole tantissima] [Sto per rivelarvi i risultati del mio test del DNA] [Il test del DNA è un regalo dei miei genitori] [Fra qualche settimana avrò i risultati] [il mio DNA sarà online] [e c'è anche una specie di social network] [Io sono Roxy e sono troppo emozionata] [perché ho i risultati del mio test del DNA] [Sono entrata nel mio account e i risultati stavano là] I risultati stavano là."
frase_ripulita = cerca_frasi_ripulite("Sto per rivelarvi i risultati del mio test")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [31]:
frase_sporca = "(risate) Cosa diceva Carlo Marx?"
frase_ripulita = cerca_frasi_ripulite("Carlo Marx?")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [33]:
frase_sporca = "A: Ascolta per ascoltare, e non per rispondere o per giudicare."
frase_ripulita = cerca_frasi_ripulite("per ascoltare, e non per rispondere o")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [37]:
frase_sporca = """
(Applausi) Un altro pregiudizio riguarda la carrozzina: quante volte ho sentito il termine <i>costretta 
sulla carrozzina</i>: è terrificante come questa espressione contenga in sé così tanta ignoranza, e come 
vada ad alimentare quel senso di pietismo che invece noi vogliamo sconfiggere.
"""

frase_ripulita = cerca_frasi_ripulite("è terrificante come questa espressione contenga")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [47]:
frase_sporca = """
A.R.: Gli strumenti di comunicazione vanno da strumenti semplici a strumenti tecnologici.
"""

frase_ripulita = cerca_frasi_ripulite("strumenti semplici a strumenti ")


lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

"No ragazzi ce ne vuole tantissima Sto per rivelarvi i risultati del mio test del DNA Il test del DNA è un regalo dei miei genitori Fra qualche settimana avrò i risultati il mio DNA sarà online e c'è anche una specie di social network Io sono Roxy e sono troppo emozionata perché ho i risultati del mio test del DNA Sono entrata nel mio account e i risultati stavano là I risultati stavano là."

In [22]:
frase_sporca = """
Quello che fa un artista di computer grafica, normalmente, è aprire il suo computer, aprire Maya che è un 
software come gli altri, 3D Studio Max o altri, schiaccia un bottone con scritto <i>Create</i>, dopo questo 
bottone schiaccia <i>Polygon</i>, <i>Cube</i> e <i>Muovi Vertice</i> - sono comandi veri del software; lo rifà 
un milione di volte, tutto il giorno rifà questa cosa, e alla fine ottiene questo.
"""
frase_ripulita = cerca_frasi_ripulite("Quello che fa un artista di computer grafica, normalmente, è aprire il suo computer")

lista_frasi = popola_lista_frasi_ripulite(frase_sporca, frase_ripulita, lista_frasi)

In [48]:
lista_frasi

[("(Applausi) (Musica di chitarra) ♪ Il venditore afgano ha visto esploder gambe ♪ ♪ Anche di suo fratello e odia perché ne ha bisogno ♪ ♪ Il giovane italiano pensa che sia un coglione ♪ ♪ Che questo non ha senso, non è valida ragione ♪ ♪ Peccato che domenica ha infilato sei centimetri di lama ♪ ♪ Dentro a un petto che batteva per un'altra squadra ♪ ♪ Che ha incrociato la sua strada e non doveva ♪ ♪ Che sgarro imperdonabile ♪ ♪ E Irina vende il corpo, ma solo più da tergo ♪ ♪ Non vuole occhi davanti a far da specchio del suo inferno ♪ ♪ E tu che a malapena hai il tuo smalto da abbinare ♪ ♪ Pensi di capire?",
  "Il venditore afgano ha visto esploder gambe Anche di suo fratello e odia perché ne ha bisogno Il giovane italiano pensa che sia un coglione Che questo non ha senso, non è valida ragione Peccato che domenica ha infilato sei centimetri di lama Dentro a un petto che batteva per un'altra squadra Che ha incrociato la sua strada e non doveva Che sgarro imperdonabile E Irina vende il c

In [63]:
nuova_lista_frasi_con_id = []

for i, tupla in zip(range(1, len(lista_frasi) +1 ), lista_frasi):
    l_temp = [i]
    l_tup = list(tupla)
    l_temp.extend(l_tup)
    l_temp = tuple(l_temp)
    
    nuova_lista_frasi_con_id.append(l_temp)

In [64]:
nuova_lista_frasi_con_id

[(1,
  "(Applausi) (Musica di chitarra) ♪ Il venditore afgano ha visto esploder gambe ♪ ♪ Anche di suo fratello e odia perché ne ha bisogno ♪ ♪ Il giovane italiano pensa che sia un coglione ♪ ♪ Che questo non ha senso, non è valida ragione ♪ ♪ Peccato che domenica ha infilato sei centimetri di lama ♪ ♪ Dentro a un petto che batteva per un'altra squadra ♪ ♪ Che ha incrociato la sua strada e non doveva ♪ ♪ Che sgarro imperdonabile ♪ ♪ E Irina vende il corpo, ma solo più da tergo ♪ ♪ Non vuole occhi davanti a far da specchio del suo inferno ♪ ♪ E tu che a malapena hai il tuo smalto da abbinare ♪ ♪ Pensi di capire?",
  "Il venditore afgano ha visto esploder gambe Anche di suo fratello e odia perché ne ha bisogno Il giovane italiano pensa che sia un coglione Che questo non ha senso, non è valida ragione Peccato che domenica ha infilato sei centimetri di lama Dentro a un petto che batteva per un'altra squadra Che ha incrociato la sua strada e non doveva Che sgarro imperdonabile E Irina vende

In [49]:
from tabulate import tabulate

In [65]:
print(tabulate(nuova_lista_frasi_con_id, headers=['N.', 'Frasi originali', 'Frasi ripulite'], tablefmt='latex'))

\begin{tabular}{rll}
\hline
   N. & Frasi originali                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    & Frasi ripulite                                                                                                                                                                                                                                                                                                                                                 